In [ ]:
%%html
<h1> Set up Jira Client</h1>

In [ ]:
#The following packages must be installed after anaconda is installed. They are commented off here.
#!pip install jira
#!pip install numpy
#!pip install pandas
#!pip install xslwriter
#!pip install json
#!pip intsall datetime
#!pip install functools

In [ ]:
from jira import JIRA
import numpy as np
import pandas as pd
import xlsxwriter
import json
from datetime import datetime
from datetime import timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

json_data_file = domain = domain = username = password = None
cpath = "./jira.json"

while not json_data_file:
    try:
        json_data_file = open(cpath)
    except FileNotFoundError:
        cpath = input('Directory Path of jira.json: ')
        cpath = cpath + '/jira.json'

data = json.load(json_data_file)
username = data['auth']['username']
password = data['auth']['password']
bugqueryadd = data['bugqueryadd']
epicqueryadd = data['epicqueryadd']
storyqueryadd = data['storyqueryadd']
domain = data['domain']
columns = data['columns']
fields = data['fields']
outfile = data['outfile']
        
#if not domain:
#    domain = input("Jira Domain (e.g https://XXX:PPP/jira): ")

#Only username and password will be accepted outside of the file
if not username:
    username = input("Username: ")

if not password:
    password = getpass.getpass("Password: ")
   
def get_jira_client(domain, username, password):
    options = {'server': domain}
    return JIRA(options, basic_auth=(username, password))
    
writer = pd.ExcelWriter(outfile)
jira = get_jira_client(domain, username, password)

In [ ]:
#Important dates/labels that set the baseline for this run
qtrStart = '2018-07-04'
qtrEnd = '2018-09-25'
qtrStartDate = pd.to_datetime(qtrStart, format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
qtrEndDate = pd.to_datetime(qtrEnd, format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
relp = 'R18'
reln = 'R19'

#for relp:
#last deadline for inserting stories
releaseStoryDeadline = datetime(2018, 7, 24)
#monitoring deadline for removing stories
releaseStoryRemovalMonitoringStart = datetime(2018, 6, 12)
releaseStoryRemovalMonitoringEnd = datetime(2018, 9, 11)

bins = [datetime(2018, 6, 19), datetime(2018, 7, 3), datetime(2018, 7, 17), 
        datetime(2018, 7, 31), datetime(2018, 8, 14), datetime(2018, 8, 28), datetime(2018, 9, 11), 
        datetime(2018, 9, 25), datetime(2018, 10, 9)]
binLabels = ['reg-sprint-24', 'r18-sprint-25', 'r18-sprint-26', 
             'r18-sprint-27', 'reg-sprint-28', 
          'reg-sprint-29', 'r19-sprint-30', 'r19-sprint-31']


In [ ]:
%%html
<h1> Load Stories, Epics and Bugs</h1>

In [ ]:
epics = jira.search_issues('type=epic and ' + epicqueryadd, json_result=True, maxResults=20000, fields = fields)

In [ ]:
stories = jira.search_issues('type=story and ' + storyqueryadd, json_result=True, maxResults=20000, fields = fields, expand='changelog')

In [ ]:
%%html
<h1> Set up the Dataframes for Stories and Epics</h1>

In [ ]:
#prep the stories and epics dataframes
#fix the column names
#extract comment data 
#extract all the history from stories and build all the workflow fields

for issue in stories['issues']:
    #merge the textual fields of comments, summary
    alltext = [comment['body'] for comment in issue['fields']['comment']['comments']]
    if (issue['fields']['summary'] != None):
        alltext.append(issue['fields']['summary'])
    if (issue['fields']['description'] != None):
        alltext.append(issue['fields']['description'])
    try:
        issue['fields']['textinfo'] = ' '.join(alltext)
    except TypeError:
        print(alltext)

    #for stories only, record the important parts of change log as separate columns
    
    issue['fields']['Open Set By'] = []
    issue['fields']['Approval Set By'] = []
    issue['fields']['Closed Set By'] = []
    issue['fields']['Code Review Set By'] = []
    issue['fields']['In Analysis Set By'] = []
    issue['fields']['In Progress Set By'] = []
    issue['fields']['In UI/UX Set By'] = []
    issue['fields']['Ready for Estimation Set By'] = []
    issue['fields']['Testing Set By'] = []
    issue['fields']['Resolved Set By'] = []
    issue['fields']['Reopened Set By'] = []
    
    changelog = issue['changelog']
    for history in changelog['histories']:
        for item in history['items']:
            #print (item['field'])
            if (item['field'] == 'Fix Version') and (item['fromString'] == relp):
                #a story was moved out of the current fix version?
                issue['fields']['FixVersion Change Date'] = pd.to_datetime(history['created'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
                #print(issue['key'], ' fix version changed from ', item['fromString'], ' to ', item['toString'])
            if item['field'] == 'status':
                #need to ensure if there are multiple times a certain status is updated, we capture it
                #the first or last time based on the specific status.
                timestamp = pd.to_datetime(history['created'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
                event = item['toString'] + ' ' + 'Set By'
                author = history['author']['name']
                issue['fields'][event].append((author, timestamp))
                #issue['fields'][item['toString'] + ' ' + 'Set To Date'] = history['created']
                #issue['fields'][item['toString'] + ' ' + 'Set By'] = history['author']['name']
    issue['fields']['Open Set By'] = min(issue['fields']['Open Set By'], key = lambda t: t[1]) if issue['fields']['Open Set By'] else None
    issue['fields']['Approval Set By'] = max(issue['fields']['Approval Set By'], key = lambda t: t[1]) if issue['fields']['Approval Set By'] else None
    issue['fields']['Closed Set By'] = max(issue['fields']['Closed Set By'], key = lambda t: t[1]) if issue['fields']['Closed Set By'] else None
    issue['fields']['Code Review Set By'] = min(issue['fields']['Code Review Set By'], key = lambda t: t[1]) if issue['fields']['Code Review Set By'] else None
    issue['fields']['In Analysis Set By'] = min(issue['fields']['In Analysis Set By'], key = lambda t: t[1]) if issue['fields']['In Analysis Set By'] else None
    issue['fields']['In Progress Set By'] = min(issue['fields']['In Progress Set By'], key = lambda t: t[1]) if issue['fields']['In Progress Set By'] else None
    issue['fields']['In UI/UX Set By'] = min(issue['fields']['In UI/UX Set By'], key = lambda t: t[1]) if issue['fields']['In UI/UX Set By'] else None
    issue['fields']['Ready for Estimation Set By'] = min(issue['fields']['Ready for Estimation Set By'], key = lambda t: t[1]) if issue['fields']['Ready for Estimation Set By'] else None
    issue['fields']['Testing Set By'] = min(issue['fields']['Testing Set By'], key = lambda t: t[1]) if issue['fields']['Testing Set By'] else None
    issue['fields']['Resolved Set By'] = min(issue['fields']['Resolved Set By'], key = lambda t: t[1]) if issue['fields']['Resolved Set By'] else None
    issue['fields']['Reopened Set By'] = min(issue['fields']['Reopened Set By'], key = lambda t: t[1]) if issue['fields']['Reopened Set By'] else None
    
    issue['fields']['Open Set To Date'] = issue['fields']['Open Set By'][1] if issue['fields']['Open Set By'] else None
    issue['fields']['Open Set By'] = issue['fields']['Open Set By'][0] if issue['fields']['Open Set By'] else None
    
    issue['fields']['Approval Set To Date'] = issue['fields']['Approval Set By'][1] if issue['fields']['Approval Set By'] else None
    issue['fields']['Approval Set By'] = issue['fields']['Approval Set By'][0] if issue['fields']['Approval Set By'] else None
    
    issue['fields']['Closed Set To Date'] = issue['fields']['Closed Set By'][1] if issue['fields']['Closed Set By'] else None
    issue['fields']['Closed Set By'] = issue['fields']['Closed Set By'][0] if issue['fields']['Closed Set By'] else None
    
    issue['fields']['Code Review Set To Date'] = issue['fields']['Code Review Set By'][1] if issue['fields']['Code Review Set By'] else None
    issue['fields']['Code Review Set By'] = issue['fields']['Code Review Set By'][0] if issue['fields']['Code Review Set By'] else None
    
    issue['fields']['In Analysis Set To Date'] = issue['fields']['In Analysis Set By'][1] if issue['fields']['In Analysis Set By'] else None
    issue['fields']['In Analysis Set By'] = issue['fields']['In Analysis Set By'][0] if issue['fields']['In Analysis Set By'] else None
    
    issue['fields']['In Progress Set To Date'] = issue['fields']['In Progress Set By'][1] if issue['fields']['In Progress Set By'] else None
    issue['fields']['In Progress Set By'] = issue['fields']['In Progress Set By'][0] if issue['fields']['In Progress Set By'] else None
    
    issue['fields']['In UI/UX Set To Date'] = issue['fields']['In UI/UX Set By'][1] if issue['fields']['In UI/UX Set By'] else None
    issue['fields']['In UI/UX Set By'] = issue['fields']['In UI/UX Set By'][0] if issue['fields']['In UI/UX Set By'] else None
    
    issue['fields']['Ready for Estimation Set To Date'] = issue['fields']['Ready for Estimation Set By'][1] if issue['fields']['Ready for Estimation Set By'] else None
    issue['fields']['Ready for Estimation Set By'] = issue['fields']['Ready for Estimation Set By'][0] if issue['fields']['Ready for Estimation Set By'] else None
    
    issue['fields']['Testing Set To Date'] = issue['fields']['Testing Set By'][1] if issue['fields']['Testing Set By'] else None
    issue['fields']['Testing Set By'] = issue['fields']['Testing Set By'][0] if issue['fields']['Testing Set By'] else None
    
    issue['fields']['Resolved Set To Date'] = issue['fields']['Resolved Set By'][1] if issue['fields']['Resolved Set By'] else None
    issue['fields']['Resolved Set By'] = issue['fields']['Resolved Set By'][0] if issue['fields']['Resolved Set By'] else None
    
    issue['fields']['Reopened Set To Date'] = issue['fields']['Reopened Set By'][1] if issue['fields']['Reopened Set By'] else None
    issue['fields']['Reopened Set By'] = issue['fields']['Reopened Set By'][0] if issue['fields']['Reopened Set By'] else None
    
    
for issue in epics['issues']:
    alltext = [comment['body'] for comment in issue['fields']['comment']['comments']]
    alltext.append(issue['fields']['summary'])
    #alltext.append(issue['fields']['description'])
    issue['fields']['textinfo'] = ' '.join(alltext)

epic_list = []
for epic in epics['issues']:
    epic['fields']['key'] = epic['key']
    epic_list.append(epic['fields'])

epics_df = pd.DataFrame(epic_list)

story_list = []
for story in stories['issues']:
    story['fields']['key'] = story['key']
    story_list.append(story['fields'])

stories_df = pd.DataFrame(story_list)

#replacement of custom field's by their names is only done inside the dataframe
# Fetch all fields
allfields=jira.fields()
# Make a map from field name -> field id
nameMap = {field['name']:field['id'] for field in allfields}
idMap = {field['id']:field['name'] for field in allfields}

for column in epics_df.columns:
    if ('custom' in column):
        epics_df.rename(columns={column: idMap[column]}, inplace=True)

for column in stories_df.columns:
    if ('custom' in column):
        stories_df.rename(columns={column: idMap[column]}, inplace=True)

stories_df['Team'] = stories_df['Team'].apply(lambda x: x[0].get('value') if (type(x) == list) else None)
stories_df['status'] = stories_df['status'].apply(lambda x: x.get('name'))
stories_df['reporter'] = stories_df['reporter'].apply(lambda x: x.get('name'))
stories_df['fixVersions'] = stories_df['fixVersions'].apply(lambda x: x[0]['name'] if ((type(x) == list) and x and (type(x[0]) == dict)) else None)
stories_df['Platform'] = stories_df['Platform'].apply(lambda x: x[0].get('value'))
stories_df['created'] = pd.to_datetime(stories_df['created'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
stories_df['resolution'] = stories_df['resolution'].apply(lambda x: x['name'] if type(x) == dict else None)


#insert a column for jira link
stories_df['story_link'] = '=HYPERLINK("' + domain + '/browse/' + stories_df['key'] + '","' + stories_df['key'] + '")'

#eliminate stories that are marked not needed
stories_df = stories_df[stories_df['resolution'] != 'Not Needed']

In [ ]:
%%html
<h1> Set up the Dataframes for Sprints</h1>

In [ ]:
#extract the sprint information from the sprints field and create a separate sprints-issue dataframe
#this is only possible once we have the stories dataframe

from functools import reduce

#Takes a list of sprints of the form:
#['com.atlassian.greenhopper.service.sprint.Sprint@1b7eb58a[id=519,rapidViewId=219,state=CLOSED,name=Knight Riders Sprint 2018 - 22,startDate=2018-05-23T21:16:06.149+05:30,endDate=2018-06-05T19:44:00.000+05:30,completeDate=2018-06-06T20:45:27.547+05:30,sequence=519]',
# 'com.atlassian.greenhopper.service.sprint.Sprint@2a28663d[id=542,rapidViewId=219,state=ACTIVE,name=Knight Riders Sprint 2018-23,startDate=2018-06-06T22:14:10.412+05:30,endDate=2018-06-19T20:42:00.000+05:30,completeDate=<null>,sequence=542]']
# and returns one list with a dictionary object for each sprint located. The object also contains the issue key
# the other is 
# we return a dictionary
def getSprintInfo(issueKey, sprint):
    #locate the part in square braces
    start = sprint.find('[') + 1
    end = sprint.find(']', start)
    dict_sprint = dict(x.split('=') for x in sprint[start:end].split(','))
    dict_sprint['issue_key'] = issueKey
    return dict_sprint

#we return a list of dictionaries, where each dictionary is a sprint paired with the issue.
def getSprints (issueKey, sprints):
    if type(sprints) == list:
        return [getSprintInfo(issueKey, sprint) for sprint in sprints]
    else:
        return []

x1 = []
for index, row in stories_df.iterrows():
    x1 = x1 + (getSprints(row['key'], row['Sprint']))

sprints_df =  pd.DataFrame(x1)
sprints_df['endDate'] = pd.to_datetime(sprints_df['endDate'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
sprints_df['startDate'] = pd.to_datetime(sprints_df['startDate'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
sprints_df['completeDate'] = pd.to_datetime(sprints_df['completeDate'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')

In [ ]:
%%html
<h1> Story and Epics distribution at the top level</h1>

In [ ]:
#Basic statistics before we start separating

print('No. Epics: ', epics_df['key'].unique().size)
print('No. Stories: ', stories_df['key'].unique().size)
print('No of stories without linked epics: ', sum(pd.isnull(stories_df['Epic Link'])))
print ('Stories not Closed: ', stories_df[stories_df['status'] != 'Closed']['key'].unique().size)
print ('Stories without a fixVersion: ', stories_df[stories_df['fixVersions'] == None]['key'].unique().size)

In [ ]:
#Lets find the distribution of status amongst all the fixVersions

storiesFixVersionsStatus_df = stories_df[['fixVersions', 'status', 'key']].copy()
storiesFixVersionsStatus_df.groupby(['fixVersions', 'status']).agg(['count'])

In [ ]:
stories_df[pd.notnull(stories_df['FixVersion Change Date']) & (stories_df['fixVersions'] != relp)][['key', 'Team', 'reporter', 'summary', 'FixVersion Change Date', 'fixVersions']].sort_values('Team')

In [ ]:
%%html
<h1> Stories that potentially have wrong fixVersion</h1>

In [ ]:
#Stories that in Code Review/Testing or Approval in reln need to be flagged
df = stories_df[((stories_df['fixVersions'].isin([reln, 'Backlog']) | pd.isnull(stories_df['fixVersions'])) & 
                        (stories_df['status'].isin(['Code Review', 'In Progress', 'Approval', 'Closed'])) )]
df[['key', 'status', 'fixVersions', 'summary']]

In [ ]:
#Lets remove the stories which we do not care about - not in relp or reln
stories_df = stories_df[((stories_df['fixVersions'] == relp) | (stories_df['fixVersions'] == reln))]

In [ ]:
#Did we change fixversion recently of a story that was potentially required in current release?
#Go through history and make a list of tickets that have had their fixVersion either set to relp or set to something 
#other than relp. So the From or To could be relp and we just need a date threshold.

In [ ]:
#For stories that are closed, lets find the time it took for us to go through each state completely, 
#the points of the story, the number of sprints it took, the team the story is in.
#We are ignoring the Reopen workflow.

stories_df['Analysis Duration'] = (stories_df['Ready for Estimation Set To Date'] - stories_df['created']).dt.days
stories_df['Dev Duration'] = (stories_df['Testing Set To Date'] - stories_df['Open Set To Date']).dt.days
stories_df['QA Duration'] = (stories_df['Approval Set To Date'] - stories_df['Testing Set To Date']).dt.days
stories_df['Approval Duration'] = (stories_df['Closed Set To Date'] - stories_df['Approval Set To Date']).dt.days


In [ ]:
#df = stories_df[stories_df['status'] == 'Closed']
df = stories_df
df = df[(df['Analysis Duration'] > 60) | (df['Dev Duration'] > 7) | (df['QA Duration'] > 2) | (df['Approval Duration'] > 1)]
df[['key', 'fixVersions', 'status', 'Team', 'Analysis Duration', 'Dev Duration', 'QA Duration', 'Approval Duration']]

In [ ]:
#calcuate the age of the stories in the last state it is in
now = datetime.now() + pd.Timedelta('010:30:00')
stories_df['Age In Days'] = stories_df.apply(lambda x: (now - x[x['status'] + ' Set To Date']).days, axis = 1)

In [ ]:
%%html
<h1> Merge Stories, Epics and Sprints</h1>

In [ ]:
#first merge - create the epics and stories merge
scope_df = pd.merge(epics_df, stories_df, how='right', on=None, left_on='key', right_on='Epic Link',
         left_index=False, right_index=False, sort=True,
         suffixes=('_epic', '_story'), copy=True, indicator=False,
         validate=None)

In [ ]:
#Combine the sprints with the epics + stories dataframe and we can then drop the duplicate issue_key field.

sprintsWithStoriesAndEpics_df = pd.merge(scope_df, sprints_df, how='left', on=None, left_on='key_story', right_on='issue_key',
         left_index=False, right_index=False, 
         suffixes=('_story', '_sprint'),
         copy=True, indicator=True,
         validate=None).drop(columns = ['issue_key'])

#We can drop stories that are in future sprints
sprintsWithStoriesAndEpics_df = sprintsWithStoriesAndEpics_df[sprintsWithStoriesAndEpics_df['state'] != 'FUTURE']

In [ ]:
#Lets make a list of problematic stories - 
#stories which are not in any sprints yet
#stories which are in inactive sprints
#after that we will elimiate these stories
print('Stories that are not yet unassigned to sprints: ')
print() 
sprintsWithStoriesAndEpics_df[sprintsWithStoriesAndEpics_df['_merge'] == 'left_only'][['key_story', 'reporter_story', 'summary_story', 'status_story']]

In [ ]:
#eliminiate the stories that are not assigned to sprints.
sprintsWithStoriesAndEpics_df = sprintsWithStoriesAndEpics_df[sprintsWithStoriesAndEpics_df['_merge'] != 'left_only']

In [ ]:
#find the stories which were opened more than a day later than the sprint started 
#or those were inserted after the development sprints were over

sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_df[['Team_story', 'startDate', 'state', 'Open Set To Date', 'reporter_story', 'Story Points', 'key_story', 'name', 'fixVersions_story']].copy()

sprintsWithStoriesAndEpics_dfCopy['sprintLeadTime'] = (sprintsWithStoriesAndEpics_dfCopy['Open Set To Date'] - sprintsWithStoriesAndEpics_dfCopy['startDate']).dt.days 
sprintsWithStoriesAndEpics_dfCopy['sprintCommitment'] = sprintsWithStoriesAndEpics_dfCopy['sprintLeadTime'] <= 1
#sprintsWithStoriesAndEpics_dfCopy['key_story'].unique().size
sprintsWithStoriesAndEpics_dfCopy['beyondReleaseDeadline'] = sprintsWithStoriesAndEpics_dfCopy['Open Set To Date'] >= releaseStoryDeadline

df = sprintsWithStoriesAndEpics_dfCopy[(sprintsWithStoriesAndEpics_dfCopy['sprintCommitment'] != True)|(sprintsWithStoriesAndEpics_dfCopy['beyondReleaseDeadline'] == True)].sort_values(by='key_story')
#df = sprintsWithStoriesAndEpics_dfCopy
#df = df[df['state'] == 'ACTIVE']
#write out the source data onto disk
#however we want to write only the records which are duplicates. Better idea to remove the non duplicates.
df.to_excel(writer, index=False, sheet_name='Late Commitments', freeze_panes=(1,0), columns=['Team_story', 'startDate', 'Open Set To Date', 'reporter_story', 'Story Points', 'key_story', 'name', 'sprintLeadTime', 'sprintCommitment'])
df.sort_values('key_story')

In [ ]:
#Lets remove the stories which we do not care about - closed
stories_df = stories_df[stories_df['status'] != "Closed"]
scope_df = scope_df[scope_df['status_story'] != "Closed"]
sprintsWithStoriesAndEpics_df = sprintsWithStoriesAndEpics_df[sprintsWithStoriesAndEpics_df['status_story'] != "Closed"]

In [ ]:
#select the latest sprint that the stories are in and then we can filter the ones that sprints that are closed.
sprintsWithStoriesAndEpics_df = sprintsWithStoriesAndEpics_df.loc[sprintsWithStoriesAndEpics_df.groupby("key_story")["startDate"].idxmax()]
sprintsWithStoriesAndEpics_df = sprintsWithStoriesAndEpics_df[pd.notnull(sprintsWithStoriesAndEpics_df.index)]


In [ ]:
print('Stories that have the most recent sprint inactive ')

#next step is to print the ones out that have inactive sprints and eliminate them
sprintsWithStoriesAndEpics_df[sprintsWithStoriesAndEpics_df['state'] == 'CLOSED'][['key_story', 'Epic Name', 'summary_story', 'status_story', 'name', 'startDate', 'state']]

In [ ]:
#eliminate the stories with recent inactive sprints
sprintsWithStoriesAndEpics_df = sprintsWithStoriesAndEpics_df[sprintsWithStoriesAndEpics_df['state'] != 'CLOSED']

In [ ]:
#Calculate sprint Age.
#sprintsWithStoriesAndEpics_df['key_story'][165]
#There are two scenarios we have not considered - if the sprint is not active anymore, the age should be zero
#We can remove stories in inactive sprints and report them as having no sprints!
#The second case if the last status change happened earlier than sprint start date.
sprintsWithStoriesAndEpics_df['Sprint Age In Days'] = sprintsWithStoriesAndEpics_df.apply(lambda x: (now - max(x[x['status_story'] + ' Set To Date'], x['startDate'])).days, axis = 1)

In [ ]:
#List the stories with their status, age and sprint age.
sprintsWithStoriesAndEpics_df[['key_story', 'Team_story', 'fixVersions_story', 'summary_story', 'status_story', 'Age In Days', 'Sprint Age In Days', 'Open Set To Date']].sort_values(by=['status_story'], ascending = False)

In [ ]:
%%html
<h1> Calculate the Stories not having any mention of AC or Acceptance.</h1>

In [ ]:
#this is a list of strings
#scope_df['textinfo'] = scope_df['textinfo_story'] + scope_df['textinfo_epic']
scope_df['textinfo'] = scope_df['textinfo_story']

In [ ]:
scope_df['Invalid AC'] = scope_df['textinfo'].str.contains('Acceptance|AC', case = False, regex = True) == False

#write out the source data onto disk
#however we want to write only the records which are duplicates. Better idea to remove the non duplicates.
scope_df[scope_df['Invalid AC']].to_excel(writer, index=False, sheet_name='Invalid AC', freeze_panes=(1,0), columns=['Team_story', 'key_story', 'reporter_story'])


In [ ]:
invalid_ac_df = scope_df[['reporter_story', 'Invalid AC']].copy()

In [ ]:
#produce statistics for valid/invalid AC
invalid_ac_df.groupby(['reporter_story']).sum().sort_values(by=['Invalid AC'], ascending=False).head()

In [ ]:
writer.save()

In [ ]:
#Amongst the stories assigned to sprints, lets do the same analysis as above
#A story may be part of many sprints. In each case we need to print the minimums, so need to construct a new dataframe

#def ListSprintAges(df, status, threshold = 1, suffix = ''):
#    print(status, ":")
#    print()
#    df.apply(lambda x: 
#             print (x['key'+suffix], x['Sprint Age In Days']) if ((x['status'+suffix] == status) 
#                                                    and x['Sprint Age In Days'] and (x['Sprint Age In Days'] >= threshold)) else None, axis=1)
#    print()

#sprintsWithStoriesAndEpics_dfCopy = sprintsWithStoriesAndEpics_df[['key_story', 'status_story', 'Sprint Age In Days']].copy()
#sprintsWithStoriesAndEpics_dfCopy.groupby(['key_story', 'status_story' ]).agg({'min'})
#ListSprintAges(sprintsWithStoriesAndEpics_dfCopy, "Open", 3, '_story')
#ListSprintAges(sprintsWithStoriesAndEpics_dfCopy, "Approval", 3, '_story')
#ListSprintAges(sprintsWithStoriesAndEpics_df[sprintsWithStoriesAndEpics_df['_merge'] == 'both'], "In Analysis", 1, '_story')
#ListSprintAges(sprintsWithStoriesAndEpics_df[sprintsWithStoriesAndEpics_df['_merge'] == 'both'], "Ready for Estimation", 1, '_story')
#ListSprintAges(sprintsWithStoriesAndEpics_df[sprintsWithStoriesAndEpics_df['_merge'] == 'both'], "In UI/UX", 1, '_story')

In [ ]:
#For stories in each of these states, we need to determine the age
#def ListAges(df, status, threshold = 1, suffix = ''):
#    print(status, ":")
#    print()
#    df.apply(lambda x: 
#             print (x['key'+suffix], x['Age In Days']) if ((x['status'+suffix] == status) 
#                                                    and x['Age In Days'] and (x['Age In Days'] >= threshold)) else None, axis=1)
#    print()
    
#ListAges(stories_df, "Approval", 1)
#ListAges(stories_df, "In Analysis", 5)
#ListAges(stories_df, "Ready for Estimation", 5)
#ListAges(stories_df, "In UI/UX", 5)